<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/TicTacToeReinforcementLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pickle
import sys

In [2]:
rng=np.random.RandomState(123)

In [22]:
def np_tup(a):
    return tuple(tuple(r) for r in a)

In [23]:
def check_win(x):
    s=np.concatenate((np.sum(x,axis=0),np.sum(x,axis=1),np.asarray([np.sum(x.diagonal()),np.sum(np.fliplr(x).diagonal())])))
    return 1*np.any(s==3) - np.any(s==-3)

In [24]:
s={k:k for k in ['DRAW', 'P1Win', 'P2Win', 'CONTINUE']}

In [26]:
def eval_move(a,Q,player, compete=False):
    moves = np.array(np.nonzero(a==0)).T
    moves=np_tup(moves)
    if len(moves)==0:
        return s['DRAW']
    a_tup=np_tup(a)
    if (player,a_tup) not in Q:
        Q[player,a_tup]={}
    Qold=Q[player,a_tup]
    Qmove_maxval=-1e6
    Qmove_best=None
    for move in moves:
        a[move]=player
        b_tup = np_tup(a)
        b_vals = Q.get((-player,b_tup),{})
        Qoldmove=Qold.get(move,0)
        Qbmax = 0 if len(b_vals)==0 else max(b_vals.values())

        Qmove = (1-lr)*Qoldmove + lr*(player*check_win(a) - g*Qbmax)
        a[move]=0   #reset board
        Q[(player,a_tup)][move]=Qmove

        if Qmove>Qmove_maxval:
            Qmove_maxval=Qmove
            Qmove_best = move
    if Qmove_best is None:
        Qmove_best=moves[rng.randint(0,len(moves))]
    if rng.rand()<0.8 or compete:
        a[Qmove_best]=player
    else:
        a[moves[rng.randint(len(moves))]]=player

    result = check_win(a)

    if result == 1:
        return s['P1Win']
    elif result ==-1:
        return s['P2Win']
    else:
        return s['CONTINUE']


In [44]:
# Generate Q-Table by playing against itself

Q={}
lr=0.8
g = 0.9
for n_learn in range(10000):
    if n_learn % 500 == 0:
        print(n_learn)
    a=np.zeros((3,3),dtype=np.int8)
    while True:
        status=eval_move(a,Q,player=1)
        if status==s['CONTINUE']:
            status=eval_move(a,Q,player=-1)
        if not(status==s['CONTINUE']):
            break

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [ ]:
with open('Q.pickle','wb') as f:
    pickle.dump(Q,f)

In [45]:
a=np.zeros((3,3),dtype=np.int8)
print(a)
while True:
    while True:
        move = input('move (col,row):')
        y,x=[int(v)-1 for v in move.split(',')]
        if a[x,y]==0:
            break
    a[x,y]=1
    if(check_win(a) == 1):
        print('YOU WIN')
        break
    status=eval_move(a,Q,-1,compete=True)
    print(a)
    print('-'*10)
    if not(status==s['CONTINUE']):
        print(status)
        break

[[0 0 0]
 [0 0 0]
 [0 0 0]]
move (col,row):2,2
[[-1  0  0]
 [ 0  1  0]
 [ 0  0  0]]
----------


KeyboardInterrupt: ignored